In [1]:
# import de librairies
import numpy as np
import pandas as pd
import string as str
#afficher toutes les lignes
# pd.set_option('display.max_row', 20)
# Configuration

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 800)

In [2]:
# repertoire de l'utilisateur
%pwd

'C:\\Users\\Cissé-LENOVO\\Desktop\\Openclassrooms\\Data analyst\\Projet 3  Réalisez une étude de santé publique\\Jupyter\\P3_cisse_yaya\\P3_02_codedonnees'

In [3]:
# Importer le fichier fr_animaux.csv
animaux = pd.read_csv('fr_animaux.csv', sep=',')
animaux = animaux.assign(source = 'animale')
animaux = animaux[animaux.Zone != 'Chine']

# Importer le fichier fr_vegetaux.csv
vegetaux = pd.read_csv('fr_vegetaux.csv', sep=',')
vegetaux = vegetaux.assign(source = 'végétale')
vegetaux = vegetaux[vegetaux.Zone != 'Chine']

# Union de 2 dataframes animaux et vegetaux
animaux_vegetaux = pd.concat([animaux, vegetaux])
#animaux_vegetaux =animaux_vegetaux[['Code zone','Zone', 'Code Produit','Produit','Code année','Élément', 'Valeur', 'source']]

cereales = pd.read_csv('fr_céréales.csv', sep=',')
# creer la colonne is_cereal
cereales = cereales.assign(is_cereal = True)
# 
sql = animaux_vegetaux
# Pivot table
animaux_vegetaux = pd.pivot_table(animaux_vegetaux, values='Valeur', index=['Code zone', 'Zone', 'Code Produit', 'Produit', 'Année', 'source'], columns='Élément', aggfunc=np.sum)
animaux_vegetaux = animaux_vegetaux.reset_index()

population = pd.read_csv('fr_population.csv', sep=',')
population[['population']] = population[['Valeur']] * 1000
population_sans_chine = population[population.Zone != 'Chine']
population_sans_chine = population_sans_chine.rename(columns={'Zone': 'pays'})
population_sans_chine = population_sans_chine[['pays', 'population']]
population_sans_chine.head(5)

pays  population
0     Afghanistan    30552000
1  Afrique du Sud    52776000
2         Albanie     3173000
3         Algérie    39208000
4       Allemagne    82727000

In [4]:
# Worl undernourishment by country
# "Sécurité alimentaire" columns
sa_columns = ['Code zone', 'Zone', 'Élément', 'Produit', 'Année', 'Valeur']

sousalimentation = pd.read_csv("fr_sousalimentation.csv")[sa_columns]

sousalimentation.rename(
    columns={
        'Code zone': 'code_pays',
        'Zone': 'pays',
        'Valeur': 'pop_sous_alim',
        'Année': 'annee'
    },
    inplace=True
)

def year_format(year_code):
    """ From year_code format (XXXXXXX) to standard year format (XXXX) """
    
    if year_code == '2011-2013':
        return 2012

    elif year_code == '2012-2014':
        return 2013
    
    elif year_code == '2013-2015':
        return 2014
    
    elif year_code == '2014-2016':
        return 2015
    
    elif year_code == '2015-2017':
        return 2016
    
    elif year_code == '2016-2018':
        return 2017

# Apply the standard year format XXXX
sousalimentation['annee'] = sousalimentation['annee'].apply(year_format)

sousalimentation.fillna(0, inplace=True)
sousalimentation['pop_sous_alim'] = pd.to_numeric(sousalimentation['pop_sous_alim'], errors='coerce')

sousalimentation = sousalimentation.assign(pop_sous_alim =  sousalimentation['pop_sous_alim']* 1000000)

# Remove "China" (We just keep the China sub-countries)
sousalimentation = sousalimentation[sousalimentation.pays != 'Chine'].copy()

# undernourishment in 2013
sousalimentation_2013 = sousalimentation[sousalimentation.annee == 2013]

sousalimentation_2013.head(5)


code_pays            pays Élément  \
0           2     Afghanistan  Valeur   
5         202  Afrique du Sud  Valeur   
10          3         Albanie  Valeur   
15          4         Algérie  Valeur   
20         79       Allemagne  Valeur   

                                              Produit  annee  pop_sous_alim  
0   Nombre de personnes sous-alimentées (millions)...   2013      7900000.0  
5   Nombre de personnes sous-alimentées (millions)...   2013      2600000.0  
10  Nombre de personnes sous-alimentées (millions)...   2013       200000.0  
15  Nombre de personnes sous-alimentées (millions)...   2013      1700000.0  
20  Nombre de personnes sous-alimentées (millions)...   2013            0.0

In [5]:
# Rename
animaux_vegetaux.rename(columns={
    'Code zone': 'code_pays',
    'Zone': 'pays',
    'Code Produit': 'code_produit',
    'Produit': 'produit',
    'Année': 'annee',
    'source' : 'origin',
    'Aliments pour animaux': 'alim_anim',
    'Autres utilisations (non alimentaire)': 'autre_util',
    'Disponibilité alimentaire (Kcal/personne/jour)': 'dispo_alim_kcal',
    'Disponibilité alimentaire en quantité (kg/personne/an)': 'dispo_alim_quantite',
    'Disponibilité de matière grasse en quantité (g/personne/jour)': 'dispo_mat_gr',
    'Disponibilité de protéines en quantité (g/personne/jour)': 'dispo_prot',
    'Exportations - Quantité': 'exportation',
    'Importations - Quantité': 'importation',
    'Nourriture': 'nour',
    'Pertes': 'perte',
    'Production': 'production',
    'Semences': 'semence',
    'Traitement': 'traitement',
    'Variation de stock': 'variation_stock',
    'Disponibilité intérieure': 'dispo_int'
}, inplace=True)

# Left join with the population size
animaux_vegetaux = animaux_vegetaux.merge(population_sans_chine, how='left', on='pays')

# Clean
animaux_vegetaux.fillna(0, inplace=True)

# From 1000 tons to kg
animaux_vegetaux['exportation'] = animaux_vegetaux['exportation'] * 1000000
animaux_vegetaux['autre_util'] = animaux_vegetaux['autre_util'] * 1000000
animaux_vegetaux['alim_anim'] = animaux_vegetaux['alim_anim'] * 1000000
animaux_vegetaux['importation'] = animaux_vegetaux['importation'] * 1000000
animaux_vegetaux['nour'] = animaux_vegetaux['nour'] * 1000000
animaux_vegetaux['perte'] = animaux_vegetaux['perte'] * 1000000
animaux_vegetaux['production'] = animaux_vegetaux['production'] * 1000000
animaux_vegetaux['semence'] = animaux_vegetaux['semence'] * 1000000
animaux_vegetaux['traitement'] = animaux_vegetaux['traitement'] * 1000000
animaux_vegetaux['variation_stock'] = animaux_vegetaux['variation_stock'] * 1000000
animaux_vegetaux['dispo_int'] = animaux_vegetaux['dispo_int'] * 1000000
animaux_vegetaux.head(10)

code_pays     pays  code_produit           produit  annee    origin  \
0          1  Arménie          2511               Blé   2013  végétale   
1          1  Arménie          2513              Orge   2013  végétale   
2          1  Arménie          2514              Maïs   2013  végétale   
3          1  Arménie          2515            Seigle   2013  végétale   
4          1  Arménie          2516            Avoine   2013  végétale   
5          1  Arménie          2517            Millet   2013  végétale   
6          1  Arménie          2518            Sorgho   2013  végétale   
7          1  Arménie          2520  Céréales, Autres   2013  végétale   
8          1  Arménie          2531   Pommes de Terre   2013  végétale   
9          1  Arménie          2532            Manioc   2013  végétale   

     alim_anim  autre_util  dispo_alim_kcal  dispo_alim_quantite  \
0   93000000.0         0.0           1024.0               130.60   
1  137000000.0  26000000.0              0.0                 0.00   
2   96000000.0         0.0              0.0                 0.03   
3    1000000.0         0.0              1.0                 0.12   
4    4000000.0         0.0              2.0                 0.37   
5          0.0         0.0              0.0                 0.00   
6          0.0         0.0              0.0                 0.00   
7    1000000.0  12000000.0             14.0                 1.96   
8  225000000.0  16000000.0             82.0                45.10   
9          0.0         0.0              0.0                 0.00   

   dispo_mat_gr  dispo_prot    dispo_int  exportation  importation  \
0          3.60       30.52  554000000.0    1000000.0  361000000.0   
1          0.00        0.00  198000000.0          0.0    9000000.0   
2          0.00        0.01  102000000.0          0.0   82000000.0   
3          0.00        0.02    1000000.0          0.0          0.0   
4          0.03        0.09    6000000.0          0.0    1000000.0   
5          0.00        0.00          0.0          0.0          0.0   
6          0.00        0.00          0.0          0.0          0.0   
7          0.04        0.39   21000000.0          0.0    4000000.0   
8          0.12        1.94  672000000.0    9000000.0    5000000.0   
9          0.00        0.00          0.0          0.0          0.0   

          nour       perte   production      semence  traitement  \
0  389000000.0  32000000.0  312000000.0   30000000.0  10000000.0   
1          0.0  15000000.0  189000000.0   14000000.0   7000000.0   
2          0.0   7000000.0   21000000.0          0.0         0.0   
3          0.0         0.0    1000000.0          0.0         0.0   
4    1000000.0         0.0    5000000.0          0.0         0.0   
5          0.0         0.0          0.0          0.0         0.0   
6          0.0         0.0          0.0          0.0         0.0   
7    6000000.0   1000000.0   17000000.0    1000000.0         0.0   
8  134000000.0  96000000.0  661000000.0  123000000.0  78000000.0   
9          0.0         0.0          0.0          0.0         0.0   

   variation_stock  population  
0     -118000000.0     2977000  
1              0.0     2977000  
2              0.0     2977000  
3              0.0     2977000  
4              0.0     2977000  
5              0.0     2977000  
6              0.0     2977000  
7              0.0     2977000  
8       15000000.0     2977000  
9              0.0     2977000

In [6]:
animaux_vegetaux['dispo_alim_kcal_p_j'] = animaux_vegetaux['dispo_alim_kcal']
animaux_vegetaux['dispo_prot'] = animaux_vegetaux['dispo_prot']

animaux_vegetaux['dispo_alim_kcal'] = animaux_vegetaux['dispo_alim_kcal'] * animaux_vegetaux['population']
animaux_vegetaux['dispo_prot_pop'] = animaux_vegetaux['dispo_prot'] * animaux_vegetaux['population']

animaux_vegetaux['dispo_alim_kcal'].fillna(0, inplace=True)
animaux_vegetaux['dispo_prot_pop'].fillna(0, inplace=True)

display(animaux_vegetaux.head(10))

code_pays     pays  code_produit           produit  annee    origin  \
0          1  Arménie          2511               Blé   2013  végétale   
1          1  Arménie          2513              Orge   2013  végétale   
2          1  Arménie          2514              Maïs   2013  végétale   
3          1  Arménie          2515            Seigle   2013  végétale   
4          1  Arménie          2516            Avoine   2013  végétale   
5          1  Arménie          2517            Millet   2013  végétale   
6          1  Arménie          2518            Sorgho   2013  végétale   
7          1  Arménie          2520  Céréales, Autres   2013  végétale   
8          1  Arménie          2531   Pommes de Terre   2013  végétale   
9          1  Arménie          2532            Manioc   2013  végétale   

     alim_anim  autre_util  dispo_alim_kcal  dispo_alim_quantite  \
0   93000000.0         0.0     3.048448e+09               130.60   
1  137000000.0  26000000.0     0.000000e+00                 0.00   
2   96000000.0         0.0     0.000000e+00                 0.03   
3    1000000.0         0.0     2.977000e+06                 0.12   
4    4000000.0         0.0     5.954000e+06                 0.37   
5          0.0         0.0     0.000000e+00                 0.00   
6          0.0         0.0     0.000000e+00                 0.00   
7    1000000.0  12000000.0     4.167800e+07                 1.96   
8  225000000.0  16000000.0     2.441140e+08                45.10   
9          0.0         0.0     0.000000e+00                 0.00   

   dispo_mat_gr  dispo_prot    dispo_int  exportation  importation  \
0          3.60       30.52  554000000.0    1000000.0  361000000.0   
1          0.00        0.00  198000000.0          0.0    9000000.0   
2          0.00        0.01  102000000.0          0.0   82000000.0   
3          0.00        0.02    1000000.0          0.0          0.0   
4          0.03        0.09    6000000.0          0.0    1000000.0   
5          0.00        0.00          0.0          0.0          0.0   
6          0.00        0.00          0.0          0.0          0.0   
7          0.04        0.39   21000000.0          0.0    4000000.0   
8          0.12        1.94  672000000.0    9000000.0    5000000.0   
9          0.00        0.00          0.0          0.0          0.0   

          nour       perte   production      semence  traitement  \
0  389000000.0  32000000.0  312000000.0   30000000.0  10000000.0   
1          0.0  15000000.0  189000000.0   14000000.0   7000000.0   
2          0.0   7000000.0   21000000.0          0.0         0.0   
3          0.0         0.0    1000000.0          0.0         0.0   
4    1000000.0         0.0    5000000.0          0.0         0.0   
5          0.0         0.0          0.0          0.0         0.0   
6          0.0         0.0          0.0          0.0         0.0   
7    6000000.0   1000000.0   17000000.0    1000000.0         0.0   
8  134000000.0  96000000.0  661000000.0  123000000.0  78000000.0   
9          0.0         0.0          0.0          0.0         0.0   

   variation_stock  population  dispo_alim_kcal_p_j  dispo_prot_pop  
0     -118000000.0     2977000               1024.0      90858040.0  
1              0.0     2977000                  0.0             0.0  
2              0.0     2977000                  0.0         29770.0  
3              0.0     2977000                  1.0         59540.0  
4              0.0     2977000                  2.0        267930.0  
5              0.0     2977000                  0.0             0.0  
6              0.0     2977000                  0.0             0.0  
7              0.0     2977000                 14.0       1161030.0  
8       15000000.0     2977000                 82.0       5775380.0  
9              0.0     2977000                  0.0             0.0

In [7]:
# Add the column "is_undernourishment" to the main DF

temp_df = pd.DataFrame(sousalimentation_2013, columns=['code_pays', 'sousalimentation', 'est_sousalimente'])
animaux_vegetaux = animaux_vegetaux.merge(temp_df, how='left', on='code_pays')
animaux_vegetaux.head(5)

code_pays     pays  code_produit produit  annee    origin    alim_anim  \
0          1  Arménie          2511     Blé   2013  végétale   93000000.0   
1          1  Arménie          2513    Orge   2013  végétale  137000000.0   
2          1  Arménie          2514    Maïs   2013  végétale   96000000.0   
3          1  Arménie          2515  Seigle   2013  végétale    1000000.0   
4          1  Arménie          2516  Avoine   2013  végétale    4000000.0   

   autre_util  dispo_alim_kcal  dispo_alim_quantite  dispo_mat_gr  dispo_prot  \
0         0.0     3.048448e+09               130.60          3.60       30.52   
1  26000000.0     0.000000e+00                 0.00          0.00        0.00   
2         0.0     0.000000e+00                 0.03          0.00        0.01   
3         0.0     2.977000e+06                 0.12          0.00        0.02   
4         0.0     5.954000e+06                 0.37          0.03        0.09   

     dispo_int  exportation  importation         nour       perte  \
0  554000000.0    1000000.0  361000000.0  389000000.0  32000000.0   
1  198000000.0          0.0    9000000.0          0.0  15000000.0   
2  102000000.0          0.0   82000000.0          0.0   7000000.0   
3    1000000.0          0.0          0.0          0.0         0.0   
4    6000000.0          0.0    1000000.0    1000000.0         0.0   

    production     semence  traitement  variation_stock  population  \
0  312000000.0  30000000.0  10000000.0     -118000000.0     2977000   
1  189000000.0  14000000.0   7000000.0              0.0     2977000   
2   21000000.0         0.0         0.0              0.0     2977000   
3    1000000.0         0.0         0.0              0.0     2977000   
4    5000000.0         0.0         0.0              0.0     2977000   

   dispo_alim_kcal_p_j  dispo_prot_pop  sousalimentation  est_sousalimente  
0               1024.0      90858040.0               NaN               NaN  
1                  0.0             0.0               NaN               NaN  
2                  0.0         29770.0               NaN               NaN  
3                  1.0         59540.0               NaN               NaN  
4                  2.0        267930.0               NaN               NaN

# Question 15 : Proposez une clé primaire pertinente pour cette table.
# Une table appelée population, contenant la population de chaque pays pour 2013. Elle devra contenir 4 colonnes : pays, code_pays, annee, population.


In [8]:
# La clé primaire choisie pour la table population est code_pays

In [9]:

population = animaux_vegetaux[['code_pays', 'pays', 'annee', 'population']]
#population = population.rename(columns={"code_zone": "code_pays", "zone":"pays"})

population.head(5)

code_pays     pays  annee  population
0          1  Arménie   2013     2977000
1          1  Arménie   2013     2977000
2          1  Arménie   2013     2977000
3          1  Arménie   2013     2977000
4          1  Arménie   2013     2977000

In [10]:
population.to_csv("population.csv", index = False)

# Question 16 : Proposez une clé primaire pertinente pour cette table.
# Une table appelée dispo_alim contenant pour chaque pays et pour chaque produit en 2013, les informations #suivantes:
        la nature du produit (deux valeurs possibles : “animal” ou “végétal”)
        disponibilité alimentaire en tonnes
        disponibilité alimentaire en Kcal/personne/jour
        disponibilité alimentaire de protéines en g/personne/jour
        disponibilité alimentaire de matières grasses en g/personne/jour
    Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, origin, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr .


In [11]:
# La clé primaire choisie pour la table dispo_alim est code_pays


In [12]:
dispo_alim = animaux_vegetaux[[ 'code_pays','pays', 'code_produit', 'produit', 'annee', 'origin', 'nour', 'dispo_alim_kcal_p_j', 'dispo_prot', 'dispo_mat_gr']]
#dispo_alim.rename(columns={'zone': 'pays', 'code_zone': 'code_pays', 'source': 'origin','dispo_prot_p_j': 'dispo_prot', 'dispo_alim_grasse': 'dispo_mat_gr'})
dispo_alim

code_pays     pays  code_produit                  produit  annee  \
0              1  Arménie          2511                      Blé   2013   
1              1  Arménie          2513                     Orge   2013   
2              1  Arménie          2514                     Maïs   2013   
3              1  Arménie          2515                   Seigle   2013   
4              1  Arménie          2516                   Avoine   2013   
...          ...      ...           ...                      ...    ...   
15600        276   Soudan          2763      Poissons Pelagiques   2013   
15601        276   Soudan          2764  Poissons Marins, Autres   2013   
15602        276   Soudan          2805         Riz (Eq Blanchi)   2013   
15603        276   Soudan          2848       Lait - Excl Beurre   2013   
15604        276   Soudan          2899             Miscellanees   2013   

         origin          nour  dispo_alim_kcal_p_j  dispo_prot  dispo_mat_gr  
0      végétale  3.890000e+08               1024.0       30.52          3.60  
1      végétale  0.000000e+00                  0.0        0.00          0.00  
2      végétale  0.000000e+00                  0.0        0.01          0.00  
3      végétale  0.000000e+00                  1.0        0.02          0.00  
4      végétale  1.000000e+06                  2.0        0.09          0.03  
...         ...           ...                  ...         ...           ...  
15600   animale  0.000000e+00                  0.0        0.00          0.00  
15601   animale  5.000000e+06                  0.0        0.04          0.01  
15602  végétale  4.800000e+07                 14.0        0.28          0.05  
15603   animale  5.892000e+09                315.0       15.75         20.01  
15604  végétale  0.000000e+00                  7.0        1.47          0.01  

[15605 rows x 10 columns]

In [13]:
dispo_alim.to_csv("dispo_alim.csv", index = False)

# Question 17 : Proposez une clé primaire pertinente pour cette table. 
# Une table appelée equilibre_prod contenant pour chaque pays et pour chaque produit en 2013, les quantités suivantes :
        disponibilité intérieure
        aliments pour animaux
        semences
        pertes
        transformés
        nourriture
        autres utilisations
    Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations.


In [14]:
# La clé primaire choisie pour la table equilibre_prod est code_produit


In [15]:
equilibre_prod = animaux_vegetaux[['code_pays', 'pays','code_produit', 'produit', 'annee', 'dispo_int', 'alim_anim', 'semence', 'perte', 'traitement', 'nour', 'autre_util']]
equilibre_prod = equilibre_prod.rename(columns={'alim_anim': 'alim_ani', 'semence' : 'semences', 'perte': 'pertes', 'traitement': 'transfo', 'nour': 'nourriture', 'autre_util': 'autres_utilisations'})
equilibre_prod

code_pays     pays  code_produit                  produit  annee  \
0              1  Arménie          2511                      Blé   2013   
1              1  Arménie          2513                     Orge   2013   
2              1  Arménie          2514                     Maïs   2013   
3              1  Arménie          2515                   Seigle   2013   
4              1  Arménie          2516                   Avoine   2013   
...          ...      ...           ...                      ...    ...   
15600        276   Soudan          2763      Poissons Pelagiques   2013   
15601        276   Soudan          2764  Poissons Marins, Autres   2013   
15602        276   Soudan          2805         Riz (Eq Blanchi)   2013   
15603        276   Soudan          2848       Lait - Excl Beurre   2013   
15604        276   Soudan          2899             Miscellanees   2013   

          dispo_int     alim_ani    semences      pertes     transfo  \
0      5.540000e+08   93000000.0  30000000.0  32000000.0  10000000.0   
1      1.980000e+08  137000000.0  14000000.0  15000000.0   7000000.0   
2      1.020000e+08   96000000.0         0.0   7000000.0         0.0   
3      1.000000e+06    1000000.0         0.0         0.0         0.0   
4      6.000000e+06    4000000.0         0.0         0.0         0.0   
...             ...          ...         ...         ...         ...   
15600  0.000000e+00          0.0         0.0         0.0         0.0   
15601  5.000000e+06          0.0         0.0         0.0         0.0   
15602  4.900000e+07          0.0   1000000.0         0.0         0.0   
15603  5.892000e+09          0.0         0.0         0.0         0.0   
15604  0.000000e+00          0.0         0.0         0.0         0.0   

         nourriture  autres_utilisations  
0      3.890000e+08                  0.0  
1      0.000000e+00           26000000.0  
2      0.000000e+00                  0.0  
3      0.000000e+00                  0.0  
4      1.000000e+06                  0.0  
...             ...                  ...  
15600  0.000000e+00                  0.0  
15601  5.000000e+06                  0.0  
15602  4.800000e+07                  0.0  
15603  5.892000e+09                  0.0  
15604  0.000000e+00                  0.0  

[15605 rows x 12 columns]

In [16]:
equilibre_prod.to_csv("equilibre_prod.csv", index = False)

# Question 18 : Vous vous en doutez... proposez encore une fois une clé primaire pertinente pour cette table !
Une table appelée sous_nutrition, contenant le nombre de personnes en sous-alimentation pour chaque pays en 2013. Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.

In [17]:
# La clé primaire choisie pour la table sous-nutrition est code_pays

In [18]:
sous_nutrition = sousalimentation_2013[['code_pays','pays', 'annee', 'pop_sous_alim']]
sous_nutrition = sous_nutrition.rename(columns={'pop_sous_alim': 'nb_personnes'})
sous_nutrition = sous_nutrition.fillna(0)
sous_nutrition.head(10)

code_pays                pays  annee  nb_personnes
0           2         Afghanistan   2013     7900000.0
5         202      Afrique du Sud   2013     2600000.0
10          3             Albanie   2013      200000.0
15          4             Algérie   2013     1700000.0
20         79           Allemagne   2013           0.0
25          6             Andorre   2013           0.0
30          7              Angola   2013     8100000.0
35          8  Antigua-et-Barbuda   2013           0.0
40        194     Arabie saoudite   2013     1600000.0
45          9           Argentine   2013     1500000.0

In [19]:
sous_nutrition.to_csv("sous_nutrition.csv", index = False)